# Results visualization
## imports

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

## Load results and sanity check

In [2]:
results = []
for path in Path('results').glob('*.csv'):
    df = pd.read_csv(path)
    df['file'] = path.name
    results.append(df)

results = pd.concat(results)

In [3]:
results

,pretrain_dur,finetune_phase1_dur,finetune_phase2_dur,pretrain_n_epochs,finetune_phase1_n_epochs,finetune_phase2_n_epochs,pretrain_train_size,pretrain_valid_size,finetune_phase1_train_size,finetune_phase1_valid_size,...,finetune_subject,finetune_scheme,finetune_fold,pretrain_train_acc,pretrain_valid_acc,finetune_phase1_train_acc,finetune_phase1_valid_acc,finetune_phase2_valid_acc,test_acc,file
0,50.338649,0.684618,0.209419,143,29,36,4620,519,480,131,...,8,Scheme 1,0,0.543939,0.500963,0.677083,0.740458,0.740458,0.65000,pre-BNCI2014_001_fin-Schirrmeister2017_sub-8.csv
1,50.338649,3.736437,1.792514,143,152,212,4620,519,480,131,...,8,Scheme 1,1,0.543939,0.500963,0.785417,0.656489,0.793893,0.72500,pre-BNCI2014_001_fin-Schirrmeister2017_sub-8.csv
2,50.338649,0.973701,0.627491,143,40,61,4620,519,480,131,...,8,Scheme 1,2,0.543939,0.500963,0.731250,0.679389,0.725191,0.65000,pre-BNCI2014_001_fin-Schirrmeister2017_sub-8.csv
3,50.338649,1.522235,0.782921,143,62,88,4620,519,480,131,...,8,Scheme 1,3,0.543939,0.500963,0.760417,0.717557,0.801527,0.68125,pre-BNCI2014_001_fin-Schirrmeister2017_sub-8.csv
4,50.338649,0.871628,0.362048,143,35,47,4620,519,480,131,...,8,Scheme 1,4,0.543939,0.500963,0.725000,0.687023,0.740458,0.68125,pre-BNCI2014_001_fin-Schirrmeister2017_sub-8.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,50.338649,8.533252,0.325035,143,170,175,4620,519,660,176,...,12,Scheme 4,0,0.543939,0.500963,0.865152,0.875000,0.897727,0.76250,pre-BNCI2014_001_fin-Schirrmeister2017_sub-12.csv
16,50.338649,9.719258,0.260761,143,200,204,4620,519,660,176,...,12,Scheme 4,1,0.543939,0.500963,0.878788,0.892045,0.892045,0.79375,pre-BNCI2014_001_fin-Schirrmeister2017_sub-12.csv
17,50.338649,4.648522,0.387688,143,94,100,4620,519,660,176,...,12,Scheme 4,2,0.543939,0.500963,0.859091,0.846591,0.869318,0.76250,pre-BNCI2014_001_fin-Schirrmeister2017_sub-12.csv
18,50.338649,18.205695,0.457421,143,371,378,4620,519,660,176,...,12,Scheme 4,3,0.543939,0.500963,0.898485,0.880682,0.886364,0.81250,pre-BNCI2014_001_fin-Schirrmeister2017_sub-12.csv


In [4]:
# check the number of results
results.groupby(
    # ['pretrain_dataset', 'finetune_dataset', 'finetune_subject']
    ['pretrain_dataset', 'finetune_dataset']  # n_subjects * 5 folds * 4 schemes
).test_acc.count()

pretrain_dataset   finetune_dataset 
BNCI2014_001       BNCI2014_001         180
                   BNCI2014_004         180
                   Schirrmeister2017    280
BNCI2014_004       BNCI2014_001         180
                   BNCI2014_004         180
                   Schirrmeister2017    280
Lee2019_MI         BNCI2014_001         180
                   BNCI2014_004         180
                   Schirrmeister2017    280
PhysionetMI        BNCI2014_001         180
                   BNCI2014_004         180
                   Schirrmeister2017    280
Schirrmeister2017  BNCI2014_001         180
                   BNCI2014_004         180
                   Schirrmeister2017    280
Name: test_acc, dtype: int64

In [7]:
# check that all experiments logged the same metrics for the same pretraining phase
assert (results.groupby('pretrain_dataset')[[
    'pretrain_dur',
    'pretrain_n_epochs',
    'pretrain_train_size',
    'pretrain_valid_size',
    'pretrain_dataset',
    'pretrain_train_acc',
    'pretrain_valid_acc',
]].nunique() == 1).all().all()

In [15]:
# check amount of data in the different subsets
# TODO inconsistent with Xie2023
print(
    results.groupby('finetune_dataset')[['finetune_phase1_train_size', 'finetune_phase1_valid_size', 'test_size']].agg(
        ['mean', 'std', 'min', 'max']))

                  finetune_phase1_train_size                       \
                                        mean        std  min  max   
finetune_dataset                                                    
BNCI2014_001                           180.0    0.00000  180  180   
BNCI2014_004                           300.0    0.00000  300  300   
Schirrmeister2017                      600.0  113.42986  240  660   

                  finetune_phase1_valid_size                       test_size  \
                                        mean        std min  max        mean   
finetune_dataset                                                               
BNCI2014_001                       58.000000   0.000000  58   58  288.000000   
BNCI2014_004                       81.777778   2.741263  80   88  315.555556   
Schirrmeister2017                 160.714286  30.204044  64  180  160.000000   

                                        
                         std  min  max  
finetune_dataset      

## Durations of the experiment phases

In [8]:
# Pre-training
results.groupby('pretrain_dataset').first().pretrain_dur

pretrain_dataset
BNCI2014_001          50.338649
BNCI2014_004          42.910408
Lee2019_MI            31.089299
PhysionetMI           54.676333
Schirrmeister2017    328.381266
Name: pretrain_dur, dtype: float64

In [16]:
# Fine-tuning
# finetuning longer on Schirrmeister
results.groupby(['finetune_dataset', 'finetune_scheme'])[['finetune_phase1_dur', 'finetune_phase2_dur']].agg(
    ['mean', 'std'])

finetune_phase1_dur            \
                                                 mean       std   
finetune_dataset  finetune_scheme                                 
BNCI2014_001      Scheme 1                   0.804482  0.619670   
                  Scheme 2                   1.525600  1.006604   
                  Scheme 3                   1.396093  0.845244   
                  Scheme 4                   1.693155  1.083675   
BNCI2014_004      Scheme 1                   0.979419  0.957452   
                  Scheme 2                   1.639533  1.426904   
                  Scheme 3                   1.498243  1.327145   
                  Scheme 4                   1.671008  1.436427   
Schirrmeister2017 Scheme 1                   3.009236  2.780488   
                  Scheme 2                   6.273115  4.278509   
                  Scheme 3                   5.385614  4.004818   
                  Scheme 4                   6.602770  4.975457   

                                  finetune_phase2_dur             
                                                 mean        std  
finetune_dataset  finetune_scheme                                 
BNCI2014_001      Scheme 1                   0.588451   0.491205  
                  Scheme 2                   0.764120   0.573649  
                  Scheme 3                   0.669113   0.420636  
                  Scheme 4                   0.804911   0.557677  
BNCI2014_004      Scheme 1                   1.295326   8.956946  
                  Scheme 2                   1.001778   1.230883  
                  Scheme 3                   0.872306   1.394030  
                  Scheme 4                   0.903049   0.933934  
Schirrmeister2017 Scheme 1                   2.399558  11.536148  
                  Scheme 2                   1.888961   1.774808  
                  Scheme 3                   1.515661   1.375686  
                  Scheme 4                   1.683208   1.533563

## Scores

In [18]:
# main test results
results.groupby(['pretrain_dataset', 'finetune_dataset', 'finetune_scheme']).test_acc.agg(['mean', 'std'])

mean       std
pretrain_dataset  finetune_dataset  finetune_scheme                    
BNCI2014_001      BNCI2014_001      Scheme 1         0.502083  0.083745
                                    Scheme 2         0.530170  0.093548
                                    Scheme 3         0.531713  0.095271
                                    Scheme 4         0.532485  0.096558
                  BNCI2014_004      Scheme 1         0.682014  0.130091
                                    Scheme 2         0.704593  0.131718
                                    Scheme 3         0.701151  0.133244
                                    Scheme 4         0.722778  0.150511
                  Schirrmeister2017 Scheme 1         0.579375  0.094117
                                    Scheme 2         0.667411  0.089956
                                    Scheme 3         0.662232  0.093671
                                    Scheme 4         0.676518  0.089518
BNCI2014_004      BNCI2014_001      Scheme 1         0.417593  0.108464
                                    Scheme 2         0.490664  0.112439
                                    Scheme 3         0.485494  0.116378
                                    Scheme 4         0.496759  0.113319
                  BNCI2014_004      Scheme 1         0.719821  0.133399
                                    Scheme 2         0.728978  0.128823
                                    Scheme 3         0.724990  0.139032
                                    Scheme 4         0.731865  0.137906
                  Schirrmeister2017 Scheme 1         0.523661  0.084778
                                    Scheme 2         0.629554  0.096179
                                    Scheme 3         0.624018  0.094431
                                    Scheme 4         0.655982  0.094543
Lee2019_MI        BNCI2014_001      Scheme 1         0.408642  0.087822
                                    Scheme 2         0.473225  0.120000
                                    Scheme 3         0.478318  0.116829
                                    Scheme 4         0.478009  0.113011
                  BNCI2014_004      Scheme 1         0.700308  0.127820
                                    Scheme 2         0.712718  0.131645
                                    Scheme 3         0.713393  0.129638
                                    Scheme 4         0.725813  0.141366
                  Schirrmeister2017 Scheme 1         0.534554  0.088457
                                    Scheme 2         0.615625  0.097522
                                    Scheme 3         0.606696  0.099679
                                    Scheme 4         0.651696  0.102593
PhysionetMI       BNCI2014_001      Scheme 1         0.447917  0.099084
                                    Scheme 2         0.507793  0.120411
                                    Scheme 3         0.511420  0.117799
                                    Scheme 4         0.509799  0.116182
                  BNCI2014_004      Scheme 1         0.691607  0.132479
                                    Scheme 2         0.711032  0.139432
                                    Scheme 3         0.711736  0.146867
                                    Scheme 4         0.710159  0.139691
                  Schirrmeister2017 Scheme 1         0.586518  0.081314
                                    Scheme 2         0.668036  0.091011
                                    Scheme 3         0.660804  0.094090
                                    Scheme 4         0.673929  0.090908
Schirrmeister2017 BNCI2014_001      Scheme 1         0.494059  0.088873
                                    Scheme 2         0.530941  0.092532
                                    Scheme 3         0.530556  0.103196
                                    Scheme 4         0.544753  0.097197
                  BNCI2014_004      Scheme 1         0.687530  0.133360
                                    Scheme 2         0.719831  0.150615
                         